In [39]:
import os
import time

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager


In [40]:
USERNAME = "amelia99"
PASSWORD = os.environ.get('password') 
if PASSWORD is None:
    PASSWORD = input("Please enter your password: ")

In [41]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [51]:
def get_current_title():
    try:
        return driver.find_element(By.XPATH, "/html/body/div[3]/div[3]/div[1]/div[2]/div[2]").text
    except:
        return ""

In [52]:
def login_user(username, password):
    if is_logged_in():
        return

    login_div = driver.find_element(By.XPATH, "//div[contains(text(), 'Login')]")
    login_div.click()

    time.sleep(1)

    username_input = driver.find_element(By.XPATH, "//input[@name='username']")
    username_input.send_keys(username)

    password_input = driver.find_element(By.XPATH, "//input[@name='password']")
    password_input.send_keys(password)

    login_button = driver.find_element(By.XPATH, "//input[@value='Login']")
    login_button.click()

    time.sleep(1)

    if is_logged_in():
        print(f"Successfully logged in as {username}")
    else:
        login_user(username, password)


In [53]:
def goto(url):
    if driver.current_url == url:
        return
    driver.get(url)
    time.sleep(0.1)

In [54]:
def is_logged_in():
    goto("https://farmrpg.com")

    if driver.find_element(By.XPATH, "//div[@id='logged_in_username']").get_attribute("innerHTML") == USERNAME:
        return True
    return False

In [55]:
def get_farming_link():
    goto("https://farmrpg.com/index.php")
    
    login_user(USERNAME, PASSWORD)

    return driver.find_element(By.XPATH, "//a[contains(@href, 'xfarm.php?id')]").get_attribute("href")
    

In [56]:
def harvest_everything():
    goto(FARMING_LINK)
    driver.find_element(By.XPATH, '//a[@class="button btnorange harvestallbtn"][contains(text(), "Harvest All Crops")]').click()

In [130]:
def plant_crop(crop_name):
    goto(FARMING_LINK)
    crop_select = Select(driver.find_element(By.XPATH, '//select[@class="seedid inlineinputlg"]'))

    available_crops = get_available_crops(crop_select=crop_select)

    crop_name = crop_name[0].upper() + crop_name[1:].lower()
    if "seeds" in crop_name.lower():
        crop_name = crop_name.replace("seeds","").replace("Seeds","").replace(" ","")
    crop_name += " Seeds"


    crop_info = available_crops[available_crops.name == crop_name]

    if crop_info.empty:
        print(f"Could not find crop {crop_name} in available crops")
        # TODO: implement buying
        return

    crop_select.select_by_index(crop_info.index.values[0])

    driver.find_element(By.XPATH, '//a[contains(text(), "Plant All")]').click()
    print(f'Planted {crop_info["amount"].values[0]} {crop_name}')
    


In [131]:
def get_available_crops(crop_select=None):
    goto(FARMING_LINK)
    if crop_select is None:
        crop_select = Select(driver.find_element(By.XPATH, '//select[@class="seedid inlineinputlg"]'))

    return pd.DataFrame([[o.get_attribute("data-name"), o.get_attribute("data-amt")] for o in crop_select.options], columns=["name", "amount"]).dropna()


In [92]:
login_user(USERNAME, PASSWORD)
FARMING_LINK = get_farm_link()
FARMING_LINK

'https://farmrpg.com/xfarm.php?id=343807'

In [132]:
plant_crop("Carrot")

Planted 412 Carrot Seeds
